Copyright (c) 2020, NVIDIA CORPORATION.
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline  

import pandas as pd
import numpy as np
import gc

In [2]:
#Reload the output of previous notebook

df = pd.read_pickle('tmp.pkl')
print( df.shape )
df.head()

(146256004, 24)


,hashtags,tweet_id,media,links,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,...,b_follower_count,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,id
0,NaN,0,0,NaN,NaN,2,11,1581262691,0,986,...,94,648,False,1478011810,False,0,0,0,0,0
1,83D6C79F5FCEC8D1CAD9E82C2C261611\tFFAD2DCF664C...,1,0,NaN,NaN,1,11,1581497241,1,1225,...,1139,46,False,1540395738,True,0,1581497559,0,1581497622,1
2,NaN,2,0,DDFFB4C01DB85921C3580F614575AA6D,BE4539C53C53FFABCFD232DB100C792B,2,11,1580978528,2,3016,...,780,440,False,1432084055,True,0,0,0,1581060554,2
3,NaN,3,0,NaN,NaN,1,54,1581321849,3,2121,...,1,45,False,1534313747,False,0,0,0,1581328518,3
4,NaN,4,5,NaN,NaN,2,11,1580956787,4,813505,...,171,388,False,1490166885,False,0,0,0,1580957807,4


In [20]:
%%time

df['len_hashtags'] = df['hashtags'].apply(lambda x: str(x).count('\t')+1 if not(pd.isnull(x)) else 0)
df['len_domains']  = df['domains'].apply(lambda x: str(x).count('\t')+1 if not(pd.isnull(x)) else 0)
df['len_links']    = df['links'].apply(lambda x: str(x).count('\t')+1 if not(pd.isnull(x)) else 0)

CPU times: user 4min 20s, sys: 18.2 s, total: 4min 38s
Wall time: 4min 38s


In [21]:
%%time
var = df['domains'].fillna('').values.copy()
gc.collect()

PD = {}
null = var[0]
PD[null] = [0,0]
count = 1
for vs in var:
    if vs != null:
        for v in vs.split('\t'):
            if v not in PD:
                PD[v] = [count,1]
                count +=1
            else:
                x = PD[v]
                x[1] += 1
                PD[v] = x
    else:
        x = PD[null]
        x[1] += 1
        PD[null] = x
        
len(PD)

CPU times: user 1min 2s, sys: 4.26 s, total: 1min 6s
Wall time: 1min 6s


In [22]:
list(PD.items())[:10]

[('', [0, 124468107]),
 ('BE4539C53C53FFABCFD232DB100C792B', [1, 16]),
 ('3330516EAADFD093A1C7B9DA83172DEB', [2, 5583]),
 ('8E1AE5ECC4EFDC42F77501ED9AA002ED', [3, 9548]),
 ('92D397F8E0F1E77B36B8C612C2C51E23', [4, 158685]),
 ('6D323BE93766E79BE423FAC5C28BE39B', [5, 5958]),
 ('5C6DB4F5101365C0E440A43F68EDBECD', [6, 61066]),
 ('8405A43A5DB3E450495AB2CB6C7CFB25', [7, 10367]),
 ('E91CDEC8DC7ABF30592FA024616FF970', [8, 1360727]),
 ('3896E26D12C903F0A00B6B1BE9A9BEA3', [9, 2228314])]

In [23]:
%%time
vari = []
for vs in var:
    if vs != null:
        li=[]
        lf=[]
        for v in vs.split('\t'):
            if v!='':
                li.append(PD[v][0])
                lf.append(-PD[v][1])
        vari.append( list(np.array(li)[np.argsort(lf)].astype(np.int32) ) )
    else:
        vari.append( [0] )
del PD
gc.collect()

len(vari), vari[:10]

CPU times: user 5min 31s, sys: 20.7 s, total: 5min 51s
Wall time: 5min 51s


In [24]:
#Get only the most frequent domain
df['domains'] = np.array( [v[0] for v in vari ] ).astype( np.int32 )
gc.collect()
del vari, var
gc.collect()

0

In [25]:
%%time
var = df['links'].fillna('').values.copy()
gc.collect()

PD = {}
null = var[0]
PD[null] = [0,0]
count = 1
for vs in var:
    if vs != null:
        for v in vs.split('\t'):
            if v not in PD:
                PD[v] = [count,1]
                count +=1
            else:
                x = PD[v]
                x[1] += 1
                PD[v] = x
    else:
        x = PD[null]
        x[1] += 1
        PD[null] = x
        
len(PD),list(PD.items())[:10]

CPU times: user 1min 15s, sys: 5.17 s, total: 1min 20s
Wall time: 1min 20s


In [26]:
%%time
vari = []
for vs in var:
    if vs != null:
        li=[]
        lf=[]
        for v in vs.split('\t'):
            if v!='':
                li.append(PD[v][0])
                lf.append(-PD[v][1])
        vari.append( list(np.array(li)[np.argsort(lf)].astype(np.int32) ) )
    else:
        vari.append( [0] )
del PD
gc.collect()

len(vari), vari[:10]

CPU times: user 5min 53s, sys: 27.9 s, total: 6min 21s
Wall time: 6min 15s


In [27]:
#Get only the most frequent link
df['links'] = np.array( [v[0] for v in vari ] ).astype( np.int32 )
gc.collect()
del vari, var
gc.collect()

0

In [28]:
%%time
var = df['hashtags'].fillna('').values.copy()
gc.collect()

PD = {}
null = var[0]
PD[null] = [0,0]
count = 1
for vs in var:
    if vs != null:
        for v in vs.split('\t'):
            if v not in PD:
                PD[v] = [count,1]
                count +=1
            else:
                x = PD[v]
                x[1] += 1
                PD[v] = x
    else:
        x = PD[null]
        x[1] += 1
        PD[null] = x
        
len(PD),list(PD.items())[:10]

CPU times: user 1min 37s, sys: 5.01 s, total: 1min 42s
Wall time: 1min 42s


In [29]:
%%time
vari = []
for vs in var:
    if vs != null:
        li=[]
        lf=[]
        for v in vs.split('\t'):
            if v!='':
                li.append(PD[v][0])
                lf.append(-PD[v][1])
        vari.append( list(np.array(li)[np.argsort(lf)].astype(np.int32) ) )
    else:
        vari.append( [0] )
del PD
gc.collect()

len(vari), vari[:10]

CPU times: user 8min 37s, sys: 33.5 s, total: 9min 11s
Wall time: 9min 4s


In [30]:
#Get only the most frequent hashtag
df['hashtags'] = np.array( [v[0] for v in vari ] ).astype( np.int32 )
gc.collect()
del vari, var
gc.collect()

0

In [31]:
df.tail(40)

,hashtags,tweet_id,media,links,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,...,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,id,len_hashtags,len_domains,len_links
12434798,238962,60783913,0,4286254,20,2,54,1581745564,30481,3184707,...,1322160066,False,0,0,0,0,146255964,3,1,1
12434799,0,58383254,0,0,0,2,54,1582055658,14302,1474815,...,1322160066,False,0,0,0,0,146255965,0,0,0
12434800,0,73443113,0,0,0,1,3,1581956484,221313,2301,...,1520545274,False,0,0,0,0,146255966,0,0,0
12434801,0,73443114,0,0,0,1,3,1581611956,4597687,156,...,1520545274,True,0,0,0,0,146255967,0,0,0
12434802,1089,73443115,0,0,0,1,54,1581948544,4704326,2440,...,1339719849,True,0,0,0,0,146255968,3,0,0
12434803,0,73443116,0,0,0,1,54,1581694686,13774338,61,...,1339719849,True,0,0,0,0,146255969,0,0,0
12434804,0,73443117,0,0,0,2,54,1581819568,3266917,757,...,1331963547,True,0,0,0,0,146255970,0,0,0
12434805,0,57776596,9,0,0,2,54,1581959147,21953,5450254,...,1566249731,False,0,0,0,0,146255971,0,0,0
12434806,0,58579802,0,4047785,10,2,11,1581591894,38873,309411,...,1578295645,False,0,0,0,0,146255972,0,1,1
12434807,29139,58167668,7,0,0,2,11,1582095604,90513,130826,...,1399475256,False,0,0,0,0,146255973,1,0,0


In [32]:
df.dtypes

hashtags               int32
tweet_id              uint32
media                  uint8
links                  int32
domains                int32
tweet_type             uint8
language               uint8
timestamp             uint32
a_user_id             uint32
a_follower_count      uint32
a_following_count     uint32
a_is_verified           bool
a_account_creation    uint32
b_user_id             uint32
b_follower_count      uint32
b_following_count     uint32
b_is_verified           bool
b_account_creation    uint32
b_follows_a             bool
reply                 uint32
retweet               uint32
retweet_comment       uint32
like                  uint32
id                    uint32
len_hashtags           int64
len_domains            int64
len_links              int64
dtype: object

In [33]:
train = df.iloc[ :121386431 ].copy()
test0 = df.iloc[ 121386431:(121386431+12434735) ].copy()
test1 = df.iloc[ (121386431+12434735): ].copy()

train.shape, test0.shape, test1.shape

((121386431, 27), (12434735, 27), (12434838, 27))

In [34]:
train.to_parquet( 'train-1.parquet' )
test0.to_parquet( 'test-0.parquet' )
test1.to_parquet( 'test-1.parquet' )
gc.collect()

0